In [41]:
import numpy as np
import random


class Machine:
    
    def __init__(self, vulnerable, percentValue, penetrate):
        self.vulnerable = vulnerable
        self.percentValue = percentValue
        self.penetrate = penetrate
        
    def changeVulnerable(self, vulnerable):
        self.vulnerable = vulnerable
    
    def changeCost(self, percentValue):
        self.percentValue = percentValue
        
    def changePenetrate(self, penetrate):
        self.penetrate = penetrate
        
    def pr(self):
        print(self.vulnerable, self.percentValue, self.penetrate)

        
def printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues):
    print('machine values = ', machineValues, 'vulnerable machine values = ', vulnerableMachineValues, 'important machine values =', importantMachineValues)
#получаем кол-во машин
def GetMachineValues():
    machineValues = random.randint(4, 15)
    return machineValues
#получаем кол-во уязвимых машин 
def GetVulnerableMachineValues(machineValues):
    vulnerableMachineValues = random.randint(1, machineValues)
    return vulnerableMachineValues

#получаем номер главной машины
def GetImportantMachineValues(machineValues):
    importantMachineValues = random.randint(1, machineValues)
    return importantMachineValues

#получаем процент, который занимает главная машина 
def GetPercentValue():
    percentValue = random.randint(30, 60)
    return percentValue

def GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues):
    #пока что сделаем цену cost фиксированной
    #дальше придумаем фукнцию для нее
    machineArray = []
    percentValue = GetPercentValue()
    reversePercent = (100 - percentValue) / (machineValues - 1)
    percentValue = percentValue / 100
    reversePercent = reversePercent / 100
    reversePercent = ("%.2f" % reversePercent)
    for i in range(machineValues):
        if i == (importantMachineValues - 1):
            tmp = Machine(False, percentValue, False)
        else:
            tmp = Machine(False, reversePercent, False)
        machineArray.append(tmp)
    #здесь баг - может 2 и более раз сгенерироваться
    #одинаковое число и поэтому кол-во машин будет неточным
    for i in range(0, vulnerableMachineValues):
        j = random.randint(0, machineValues - 1)
        machineArray[j].changeVulnerable(True)
    return machineArray
#возвращает число уязвимых машин, так как из-за бага
#прошлое число уязвимых != новому
def GetVulnerableMachine(machineValues, machineArray):
    vulnerableMachine = 0
    for i in range(machineValues):
        if machineArray[i].vulnerable == True:
            vulnerableMachine += 1
    return vulnerableMachine

def CostFunction(s, a, Ei):
    return -(s * Ei *Ei)/(a * Ei - 1)

def UtilityDefender(s, a, Ei, machineValues):
    # l - процент понесенных потерь машин
    tmp = 0
    for i in range(machineValues):
        tmp += CostFunction(s, a, Ei)
    return - l - tmp

def UtilityAttacker(machineValues):
    #потом придумаем как считать l
    return l
    
def AttackerStrategyProportional(machineArray):
    attackerProportionalEffort = []
    for i in range(len(machineArray)):
        tmp = machineArray[i].percentValue
        attackerProportionalEffort.append(tmp)
    return attackerProportionalEffort
        
def AttackerStrategyHigh():
    attackerHighEffort = 1
    return attackerHighEffort
        
def DefenderStrategyNothing():
    return None
    
def DefenderStrategyProportional(machineArray, Ei):
    defenderProportionalEffort = []
    for i in range(len(machineArray)):
        tmp = float(machineArray[i].percentValue) * Ei
        defenderProportionalEffort.append(tmp)
    return defenderProportionalEffort
    
def DefenderStrategyHigh(Ei):
    defenderHighEffort = Ei
    return defenderHighEffort
#P-атакующий H-защитник
def ComparisonStrategyPH(machineArray, importantMachineValues, Ei):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    defenderHighEffort = DefenderStrategyHigh(Ei)
    if attackerProportionalEffort[importantMachineValues - 1] > defenderHighEffort:
        return True
    else:
        return False
#P-атакующий P-защитник
def ComparisonStrategyPP(machineArray, Ei):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    defenderProportionalEffort = DefenderStrategyProportional(machineArray, Ei)
    count = 0
    for i in range(len(machineArray)):
        #Дописать, так как атакующий может взломать более 1 машины
        #Дописал
        if float(attackerProportionalEffort[i]) > defenderProportionalEffort[i]:
            machineArray[i].changePenetrate(True)
            count += 1
    if count > 0:
        return True
    else:
        return False
#P - атакующий N - защитник
def ComparisonStrategyPN(machineArray):
    attackerProportionalEffort = AttackerStrategyProportional(machineArray)
    #Дописать, так как атакующий может взломать более 1 машины
    #Дописал
    count = 0
    for i in range(len(machineArray)):
        if float(attackerProportionalEffort[i]) > float(machineArray[i].percentValue):
            machineArray[i].changePenetrate(True)
            count += 1
    if count > 0:
        return True
    else:
        return False
#H-атакующий P-защитник
def ComparisonStrategyHP(machineArray, Ei):
    attackerHighEffort = AttackerStrategyHigh()
    defenderProportionalEffort = DefenderStrategyProportional(machineArray, Ei)
    if attackerHighEffort > defenderProportionalEffort[importantMachineValues - 1]:
        return True
    else:
        return False
#H-атакующий H-защитник
def ComparsionStrategyHH(machineArray, Ei):
    attackerHighEffort = AttackerStrategyHigh()
    defenderHighEffort = DefenderStrategyHigh(Ei)
    if attackerHighEffort > defenderHighEffort:
        return True
    else:
        return False
#H-Атакующий N-защитник
def ComparsionStrategyHN(machineArray, importantMachineValues):
    attackerHighEffort = AttackerStrategyHigh()
    if float(attackerHighEffort) > machineArray[importantMachineValues - 1].percentValue:
        return True
    else:
        return False

EiArray = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
machineValues = GetMachineValues()
vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
importantMachineValues = GetImportantMachineValues(machineValues)
machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues)
for i in range(machineValues):
    machineArray[i].pr()
vulnerableMachine = GetVulnerableMachine(machineValues, machineArray)
print(vulnerableMachine)
for s in (0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9):
    for a in range(1, 11, 1):
        for strategyA in (1, 2):
            for strategyD in (1, 2, 3):
                for Ei in EiArray:
                    if strategyA == 1 and strategyD == 1:
                        ComparisonStrategyPH(machineArray, importantMachineValues, Ei)
                    elif strategyA == 1 and strategyD == 2:
                        ComparisonStrategyPP(machineArray, Ei)
                    elif strategyA == 1 and strategyD == 3:
                        ComparisonStrategyPN(machineArray)
                    elif strategyA == 2 and strategyD == 1:
                        ComparisonStrategyHP(machineArray, Ei)
                    elif strategyA == 2 and strategyD == 2:
                        ComparsionStrategyHH(machineArray, Ei)
                    elif strategyA == 2 and strategyD == 3:
                        ComparsionStrategyHN(machineArray, importantMachineValues)

machine values =  9 vulnerable machine values =  7 important machine values = 2
True 0.07 False
True 0.42 False
False 0.07 False
False 0.07 False
False 0.07 False
True 0.07 False
False 0.07 False
True 0.07 False
True 0.07 False
5


In [23]:
EiArray = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for Ei in EiArray:
    print(Ei, type(Ei))

0.1 <class 'float'>
0.2 <class 'float'>
0.3 <class 'float'>
0.4 <class 'float'>
0.5 <class 'float'>
0.6 <class 'float'>
0.7 <class 'float'>
0.8 <class 'float'>
0.9 <class 'float'>


In [15]:
import sys

class Machine:
    
    def __init__(self, v, cost):
        self.v = v
        self.cost = cost
        
    def change(self, v, cost):
        self.v = v
        self.cost = cost
        
    def pr(self):
        print(self.v, self.cost)

        
aM = []
cost = 13
for i in range(10):
    tmp = Machine(False, cost)
    aM.append(tmp)
for i in range(10):
    aM[i].pr()
print('_____')
for i in range(10):
    if i == 0 or i == 3 or i == 5 or i == 9:
        aM[i].change(True, 16)
    aM[i].pr()

#machine1 = Machine(True, 45)
#machine1.pr()
#print(machine1.__sizeof__())
#print(sys.getsizeof(machine1))
#print('__________')
#a = (True, 45)
#print(a.__sizeof__())
#print(sys.getsizeof(a))

False 13
False 13
False 13
False 13
False 13
False 13
False 13
False 13
False 13
False 13
_____
True 16
False 13
False 13
True 16
False 13
True 16
False 13
False 13
False 13
True 16


In [15]:
class Machine:
    
    def __init__(self, vulnerable, percentValue, penetrate):
        self.vulnerable = vulnerable
        self.percentValue = percentValue
        self.penetrate = penetrate
        
machineArray = []
for i in range(5):
    tmp = Machine(True, False, 0.3)
    machineArray.append(tmp)
a = 0.123 * machineArray[3].penetrate
print(a)

0.036899999999999995
